## 🎯📌 **LangGraph** Custom Tool Calling

In this notebook we will have a look at how to call pre-defined and custom-made tools within our LangGraph application.

### 🔑 Setting Up OpenAI API Key

Before we begin using LangGraph, you'll need to set up your `OpenAI API key`. This key is required for generating `responses` that power our LangGraph application.

#### 🌐 Getting Your API Key

1. Visit [OpenAI API Keys Page](https://platform.openai.com/api-keys)
2. Sign in or create an account if you haven't already
3. Click on "Create new secret key"

> ⚠️ **Note**: New users get `$5 in free credits`. After that, you'll need to set up billing to continue using the API.

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OPENAI_API_KEY: "
)

The `model` is important to consider here, alot of the **local models struggle using tool calling**, however this is usually outlined on the model description.

In [3]:
from langchain_openai import ChatOpenAI

# The LLM model we want to use
openai_model = "gpt-4o-mini"
# For normal accurate responses
llm = ChatOpenAI(temperature=0.0, model=openai_model, openai_api_key = os.environ["OPENAI_API_KEY"])

### 🔑 Setting Up SerpAPI key

To test both `custom` and `pre-defined` tools, we want to use `SerpAPI` as a search tool, in order to do this we need the API key.

#### 🌐 Getting Your API Key

1. Visit [SerpAPI Keys Page](https://serpapi.com/users/sign_in)
2. Sign in or create an account if you haven't already
3. Click on "Create new secret key"

In [5]:
import os
from getpass import getpass

os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") or getpass(
    "Enter SERPAPI_API_KEY: "
)

### 🔑 Setting Up Aurelio AI API key

To obtain articles from the web

#### 🌐 Getting Your API Key

1. Visit [Aurelio Platform API Keys Page](https://serpapi.com/users/sign_in)
2. Sign in or create an account if you haven't already
3. Click on "Create new secret key"

In [6]:
import sys
import os
from getpass import getpass

sys.path.append("..")


os.environ["AURELIO_API_KEY"] = os.getenv("AURELIO_API_KEY") or getpass(
    "Enter Aurelio API Key: "
)

In [7]:
from aurelio_sdk import AurelioClient

client = AurelioClient()

In [8]:
print(dir(client))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'api_key', 'base_url', 'chunk', 'embedding', 'extract_file', 'extract_url', 'get_document', 'headers', 'source', 'wait_for']


In [9]:
test_url = "https://arxiv.org/pdf/2408.15291"
try:
    response = client.extract_url(url=test_url, quality="low", chunk=True, wait=60, polling_interval=5)
    print("API key is valid.")
except Exception as e:
    print(f"API key is invalid: {e}")

API key is valid.


### Assembling our **Tools** 🛠️
We have two ways in this notebook you can attach tools:

- If you want to attach a **pre-made** tool such as SerpAPI, LangChain comes with a few tools already defined within the `load_tools` function
- If you would instead prefer to use **custom** tools we also define a handful of mathmatic operations which will also get passed into our LLM.

In [37]:
from langchain.agents import load_tools

tools = load_tools(["serpapi"])

serpapi_tool = tools[0]

In [38]:
from aurelio_sdk import ExtractResponse

def obtain_pdf_from_url(pdf_url: str) -> str:
    """
        Obtains the pdf from the pdf_url given, a properly formatted 
        one should look like: https://arxiv.org/pdf/2408.15291
        make sure that the url given is a pdf format and not text/HTML
    """
    response_pdf_url: ExtractResponse = client.extract_url(
        url=pdf_url, quality="low", chunk=True, wait=60, polling_interval=5
    )

    response_pdf_url
    content = response_pdf_url.document.content
    return content

In [39]:
def summarize_string(content: str) -> str:
    """
        This function will summarize the content passed within, 
        the content must be in string formatting.
    """
    try:
        response = llm.invoke(f"Please provide a concise summary of the following article:\n\n{content}")
        return response.content
    except Exception as e:
        return f"Error generating summary: {str(e)}"

In [40]:
import string

def create_slug(title):
    # Remove punctuation and convert to lowercase
    title = title.translate(str.maketrans("", "", string.punctuation)).lower()
    # Replace spaces with hyphens
    slug = title.replace(" ", "-")
    return slug

def extract_title(content: str) -> str:
    """
        This function will extract the title from the content passed within,
        the content must be in string formatting.
        the return will be created as a slug title
    """
    try:
        response = llm.invoke(f"Extract the title from this article. Look at the first few lines as titles are usually at the top:\n\n{content[:500]}")
        return create_slug(response.content)
    except Exception as e:
        return f"Error extracting title: {str(e)}"

In [41]:
def extract_tags(content: str) -> list[str]:
    """
        This function takes in the content as a string and outputs a bunch of tags
        that best match the content given, the output is a list of strings.
    """
    try:
        response = llm.invoke(f"Based on the content, generate relevant tags. Return only a Python list of strings.\n\nContent: {content[:1000]}")
        # Convert the string response to an actual list
        tags_string = response.content.strip()
        # Remove any markdown formatting and evaluate the string as a Python list
        tags_string = tags_string.replace('```python', '').replace('```', '')
        tags = eval(tags_string)
        return tags
    except Exception as e:
        return [f"Error extracting tags: {str(e)}"]

Now we **bind** our tools using the `bind_tools` method which passes in a list of tools which we can create down below.

In [2]:
tools = [serpapi_tool, obtain_pdf_from_url, summarize_string, extract_title]
llm_with_tools = llm.bind_tools(tools)

NameError: name 'serpapi_tool' is not defined

### 🏦 Setting Up LangGraph

Now we want to incorporate the `llm_with_tools` into our application, to do this, we will create an **assistant** node that will create tool calls and end the application if nessercary, then we create a **conditional edge** that will use the tool box we created previously, this all gets handled through the `MessageState` which sends AI messages with tool calls all included for us.

In [43]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [1]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

react_graph = builder.compile()

display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

NameError: name 'MessagesState' is not defined

### 🧪 Testing the LangGraph

Ensure that the `tools` are correctly routed to the user queries handlers, within this we can see:

- User inital query under `Human Message`
- Tool calls and args passed in under `Ai Message`
- Tool usage under `Tool Message`
- Once finished the `Ai Message` will have a final answer for the query 

In [ ]:
messages = [HumanMessage(content="Find be an PDF article online about LangChain, obtain the pdf, and generate a list of tags using only tools.")]
messages = react_graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

2025-02-05 11:39:04 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-05 11:39:07 - httpx - INFO - _client.py:1025 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [46]:
print(messages['messages'][-1].content)